# BatFit LoRA Training on Colab
This notebook installs the lightweight dependencies, optionally mounts Google Drive for artifact storage, and launches `scripts/train_lora.py` on a Colab GPU runtime. Feel free to tweak hyperparameters via environment variables before running the training cell.

In [ ]:
!pip install -q -r requirements-colab.txt

## (Optional) Mount Drive
Uncomment the following cell if you want to save checkpoints into Drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Configure env vars
Set the base model (TinyLlama by default) and adjust sequence length/epochs when experimenting.

In [ ]:
import os
os.environ['BATFIT_BASE_MODEL'] = os.environ.get('BATFIT_BASE_MODEL', 'TinyLlama/TinyLlama-1.1B-Chat-v1.0')
os.environ['BATFIT_MAX_LEN'] = os.environ.get('BATFIT_MAX_LEN', '768')
os.environ['BATFIT_EPOCHS'] = os.environ.get('BATFIT_EPOCHS', '2')
print('Base model:', os.environ['BATFIT_BASE_MODEL'])

## Launch training
Make sure this notebook is running from the project root (the cell below assumes `scripts/train_lora.py` is available).

In [ ]:
!python scripts/train_lora.py